In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import flopy
from flopy.mf6.utils import Mf6Splitter

# Example 1 Model - Run the model in parallel

In [ ]:
# model name and simulation workspace
name = "ex1"
ws = pl.Path("working/single")

# Load the base model

In [ ]:
# load the single model using flopy.mf6.MFSimulation.load()
sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=ws)

In [ ]:
# get the gwf model in the simulation for plotting
gwf = sim.get_model()

In [ ]:
# run the single model with PETSc
# set processor=1


# Load the split model

In [ ]:
# path for the split model
split_ws = pl.Path("working/split")

In [ ]:
# load the split model using flopy.mf6.MFSimulation.load
new_sim = flopy.mf6.MFSimulation.load(sim_name=name, sim_ws=split_ws)

# Change the workspace, write the model, and run in parallel

In [ ]:
# change the workspace for the parallel model
new_ws = pl.Path("working/parallel")

In [ ]:
# set the simulation path for the parallel model to new_ws using set_sim_path
new_sim.set_sim_path(new_ws)

In [ ]:
# write the split model to the parallel model work space


In [ ]:
# set the number of processors to use with the length of the model names in the split model
# name the variable nprocessors


In [ ]:
# run the parallel model
# set processors=nprocessors in run_simulation


## Get model output

### Single model

In [ ]:
# Set a select (time step, stress period) or totim that can be used for all output extractions
kstpkper = (9,2)

In [ ]:
# get the head
# name the variable head
head = gwf.output.head().get_data(kstpkper=kstpkper)

In [ ]:
# get the specific discharge
spdis = gwf.output.budget().get_data(text="DATA-SPDIS", kstpkper=kstpkper)[0]

In [ ]:
# convert specific discharge into qx, qy, and qz vectors using flopy.utils.postprocessing.get_specific_discharge()
# name the variables qx, qy, and qz
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

### Parallel model

In [ ]:
# load the split model json and create a Mf6Splitter object
json_path = pl.Path("working/split/split.json")
mfsplit = Mf6Splitter(sim)
mfsplit.load_node_mapping(new_sim, json_path)

In [ ]:
# create a dictionary for the split model head data and reconstruct a single array
head_dict = {}
for idx, modelname in enumerate(new_sim.model_names):
    mnum = int(modelname.split("_")[-1])
    h = new_sim.get_model(modelname).output.head().get_data(kstpkper=kstpkper)
    head_dict[mnum] = h
new_head = mfsplit.reconstruct_array(head_dict)    

In [ ]:
# create a dictionary for the split model specific discharge data and reconstruct a single array for qx, qy, and qz
qx_dict = {}
qy_dict = {}
qz_dict = {}
for idx, modelname in enumerate(new_sim.model_names):
    mnum = int(modelname.split("_")[-1])
    new_gwf = new_sim.get_model(modelname)
    h = new_gwf.output.head().get_data(kstpkper=kstpkper)
    spdis = new_gwf.output.budget().get_data(text="DATA-SPDIS", kstpkper=kstpkper)[0]
    qqx, qqy, qqz = flopy.utils.postprocessing.get_specific_discharge(spdis, new_gwf)
    qx_dict[mnum] = qqx
    qy_dict[mnum] = qqy
    qz_dict[mnum] = qqz
new_qx = mfsplit.reconstruct_array(qx_dict)
new_qy = mfsplit.reconstruct_array(qy_dict)
new_qz = mfsplit.reconstruct_array(qz_dict)

## Compare head results

Use numpy to evaluate the difference between the two head arrays

## Plot model results

In [ ]:
# set the model layer of interest (options are 0, 1, or 2)
plt_lay = 2

In [ ]:
# calculate the min and max head value for the single model to ensure the color scheme is the same
vmin, vmax = head[plt_lay].min(), head[plt_lay].max()
vmin, vmax

### Create a Comparison Map

In [ ]:
fig = plt.figure(figsize=(15,7))

ax = fig.add_subplot(1, 3, 1)
pmv = flopy.plot.PlotMapView(model=sim.get_model(), ax=ax, layer=plt_lay)
hp = pmv.plot_array(head, vmin=vmin, vmax=vmax)
pmv.plot_grid()
pmv.plot_vector(qx, qy)
cb = plt.colorbar(hp, ax=ax, shrink=0.75, orientation="horizontal")
ax.set_title("Single Model")

ax = fig.add_subplot(1, 3, 2)
pmv = flopy.plot.PlotMapView(model=sim.get_model(), ax=ax, layer=plt_lay)
hp = pmv.plot_array(new_head, vmin=vmin, vmax=vmax)
pmv.plot_vector(new_qx, new_qy)
pmv.plot_grid()
cb = plt.colorbar(hp, ax=ax, shrink=0.75, orientation="horizontal")   
ax.set_title("Parallel Model")

ax = fig.add_subplot(1, 3, 3)
pmv = flopy.plot.PlotMapView(model=sim.get_model(), ax=ax, layer=plt_lay)
dp = pmv.plot_array(head-new_head)
pmv.plot_grid()
cb = plt.colorbar(dp, ax=ax, shrink=0.75, orientation="horizontal")
ax.set_title("Single Model - Parallel Model");